In order to know more information about a stock option, this options calculator with Black-Scholes Model, the first widely used model for option pricing, can provide the call/put option price, d1, d2, and Greek letters. It can assist investors in establishing an option trading strategy. 

#### Certain assumptions must be made due to this calculator is modeled by Black-Scholes model. 
  * It works on European options that can only be exercised at expiration.
  * No dividends are paid out during the option’s life.
  * Stock markets are efficient. 
  * There are no transaction and commissions costs in buying the option.
  * The risk-free rate and volatility of the underlying are known and constant.
  * The returns on the underlying are normally distributed.

#### Input variables:
  * Underlying price (per share): S; 
  * Strike price of the option (per share): K;
  * Time to maturity (years): T;
  * Continuously compounding risk-free interest rate: r;
  * Volatility: sigma;

#### Output Variables:
The cumulative density function of normal distribution N(.)
- N(d2) is the risk-adjusted probability that the option will be exercised.
- N(d1) is the probability of receiving the stock at expiration of the option.

The Greek letters
- Delta: The rate of change of the option price respected to the rate of the change of underlying asset price. 
- Gamma: The rate of change of delta respected to the rate of change of underlying asset price.
- Vega: The rate of change of the option price respected to the volatility of the underlying asset. 
- Rho: The rate of the option price respected to the interest rate. 
- Theta: The rate of change of the option price respected to the passage of time.

In [6]:
## import certain packages
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame

In [7]:
# Underlying price (per share): S; 
# Strike price of the option (per share): K;
# Time to maturity (years): T;
# Continuously compounding risk-free interest rate: r;
# Volatility: sigma;

## define two functions, d1 and d2 in Black-Scholes model
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/sigma*sqrt(T)
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

## define the call options price function
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

## define the put options price function
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

## define the Call_Greeks of an option
def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))

## define the Put_Greeks of an option
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))




# Input 

In [8]:
from datetime import datetime
from pandas import DataFrame

# Function to validate positive numeric input
def validate_positive_numeric_input(prompt):
    while True:
        value = input(prompt)
        try:
            value = float(value)
            if value <= 0:
                print("Value must be positive. Please try again.")
                continue
            return value
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

# Function to validate future expiration date
def validate_future_date(prompt):
    while True:
        expiration_date = input(prompt)
        try:
            expiration_date = datetime.strptime(expiration_date, "%m-%d-%Y")
            if expiration_date <= datetime.utcnow():
                print("Expiration date must be in the future. Please try again.")
                continue
            return expiration_date
        except ValueError:
            print("Invalid date format. Please enter date in mm-dd-yyyy.")

# Input the current stock price
S = validate_positive_numeric_input("What is the current stock price? ")

# Input the strike price
K = validate_positive_numeric_input("What is the strike price? ")

# Input the expiration date and calculate time to maturity
expiration_date = validate_future_date("What is the expiration date of the options? (mm-dd-yyyy) ")
T = (expiration_date - datetime.utcnow()).days / 365

# Input the continuously compounding risk-free interest rate
r = validate_positive_numeric_input("What is the continuously compounding risk-free interest rate in percentage(%)? ")

# Input the volatility
while True:
    sigma = validate_positive_numeric_input("What is the volatility in percentage(%)? ")
    if sigma > 100:
        print("Volatility cannot be greater than 100%. Please enter a valid value.")
    else:
        break

# Create a DataFrame of the inputs
data = {'Symbol': ['S', 'K', 'T', 'r', 'sigma'],
        'Input': [S, K, T, r, sigma]}
input_frame = DataFrame(data, columns=['Symbol', 'Input'], 
                        index=['Underlying price', 'Strike price', 'Time to maturity', 
                               'Risk-free interest rate', 'Volatility'])

print(input_frame)


                        Symbol      Input
Underlying price             S  250.00000
Strike price                 K  300.00000
Time to maturity             T    0.99726
Risk-free interest rate      r    5.00000
Volatility               sigma   10.00000


## Output

In [9]:
## calculate the call / put option price and the greeks of the call / put option
r = r/100; sigma = sigma/100;
price_and_greeks = {'Call' : [bs_call(S,K,T,r,sigma), call_delta(S,K,T,r,sigma), call_gamma(S,K,T,r,sigma),call_vega(S,K,T,r,sigma), call_rho(S,K,T,r,sigma), call_theta(S,K,T,r,sigma)],
                    'Put' : [bs_put(S,K,T,r,sigma), put_delta(S,K,T,r,sigma), put_gamma(S,K,T,r,sigma),put_vega(S,K,T,r,sigma), put_rho(S,K,T,r,sigma), put_theta(S,K,T,r,sigma)]}
price_and_greeks_frame = DataFrame(price_and_greeks, columns=['Call','Put'], index=['Price', 'delta', 'gamma','vega','rho','theta'])
price_and_greeks_frame

,Call,Put
Price,1.146841,36.554762
delta,0.101513,-0.898487
gamma,0.007107,0.007107
vega,0.442970,0.442970
rho,0.241651,-2.604608
theta,-0.034325,0.108379
